In [1]:
using Clapeyron
using Plots
using CSV
using DataFrames
using LaTeXStrings
using Measures
using Metaheuristics

In [ ]:
data = CSV.read("SO2_EXP_0.5wt.csv", DataFrame; header=false)

In [ ]:
@doc """
Inputs:
    model::EoSModel
    T = 292, 313, 333 / K
    w = wt% MEA
    loading = SO2 loading

Outputs:
    partial pressure of SO2 at given T, wt % MEA & SO2 loading

"""
function partial_SO2(model::EoSModel, T, w, loading)
    
    return nothing
end 


partial_SO2

In [4]:
@doc partial_SO2

Inputs:     model::EoSModel     T = 292, 313, 333 / K     w = wt% MEA     loading = SO2 loading

outputs:     partial pressure of SO2 at given T, wt % MEA & SO2 loading
